## Sample Model

Based on the exploration, we try the following *as a starting point*.

> #### Delete the following columns
> - days_with_fog
> - direction_peak_wind_speed
> - direction_max_wind_speed
> - max_wind_speed
> - year_built
> - days_above_90F
> - days_above_110F
> - facility_type
> - site_eui (obviously, because target column)
> - For now also delete Year_Factor

> #### Impute missing values for energy_star_rating 
> Imputing is done by replacing nan by the mean <br>
> By second thoughts we do NOT impute as XGBoost infers missing values

> #### One-hot encode categorical values 
> - State_Factor
> - Building_Class

For the sample model, we use **random forest** and **xgboost**.

In [1]:
import sklearn
import xgboost as xgb
from evaluation import RMSE
from evaluation import helper_func
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import lightgbm as lgbm
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import optuna
import math
from sklearn.model_selection import cross_val_score, KFold

#### Preparing and reading data

In [2]:
data = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
submission = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/test.csv")

In [3]:
len(submission)

9705

### Feature engineering

#### delete columns

In [4]:
reduced = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")

# to_delete = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed",
#              "days_above_90F", "days_above_110F", "Year_Factor"]

to_delete_less = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed"]
# to_delete_less = ["building_class"]

def delete_cols(dataframe, columns):
    for colname in columns:
        del dataframe[colname]

delete_cols(reduced, to_delete_less)
delete_cols(submission, to_delete_less)

# # collect garbage
# gc.collect()


#### Temperature difference

In [5]:
min_temp = [string for string in reduced.columns if 'min_temp' in string]
max_temp = [string for string in reduced.columns if 'max_temp' in string]

In [6]:
below_ex = [string for string in reduced.columns if 'days_below_30F' in string]
above_ex = [string for string in reduced.columns if 'days_above_80F' in string]

In [7]:
above_ex

['days_above_80F']

In [8]:
# Per month, do max - min

for min_, max_ in zip(min_temp, max_temp):
    name = min_.split("min")[0] + "diff"
    reduced[name] = reduced[max_] - reduced[min_]
    
for min_, max_ in zip(min_temp, max_temp):
    name = min_.split("min")[0] + "diff"
    submission[name] = submission[max_] - submission[min_]

In [9]:
# days_below_10f, days above_100f

for min_, max_ in zip(below_ex, above_ex):
    name = "extreme_temp"
    reduced[name] = reduced[max_] - reduced[min_]
    
for min_, max_ in zip(below_ex, above_ex):
    name = "extreme_temp"
    submission[name] = submission[max_] - submission[min_]

In [10]:
# delete_cols(reduced, min_temp)
# delete_cols(reduced, max_temp)
# delete_cols(submission, min_temp)
# delete_cols(submission, max_temp)

#### Impute variables 

In [11]:
# impute with mean, ensures highest correlation

reduced['energy_star_rating'] = reduced['energy_star_rating'].fillna(56.0)
submission['energy_star_rating'] = submission['energy_star_rating'].fillna(56.0)
# median 76.0 no
# 63 random
# 56 works best
reduced['year_built'] = reduced['year_built'].fillna(reduced['year_built'].median())
submission['year_built'] = submission['year_built'].fillna(submission['year_built'].median())

In [12]:
# fillmedian = ["direction_max_wind_speed", "direction_peak_wind_speed", "days_with_fog", "max_wind_speed"]

# for i in fillmedian:
#     reduced[i] = reduced[i].fillna(reduced[i].median())
#     submission[i] = submission[i].fillna(submission[i].median())

In [13]:
# test which value enhances highest correlation still
# highest_corr = []

# for i in data['energy_star_rating'].unique():
#     d = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
#     d['energy_star_rating'] = d['energy_star_rating'].fillna(i)
#     correlations = helper_func.get_correlation(d, 'site_eui', 0)[1]
#     highest_corr.append((i, correlations[1]))

#### Create new columns

In [14]:
# below compute sum of below 30 and sum of above 80
days_ = [string for string in reduced.columns if 'days_' in string]

In [15]:
# # below compute sum of below 30 and sum of above 80
# reduced['d_below_30_F'] = reduced[list(reduced.filter(regex='days_below_'))].sum(axis=1)
# reduced['d_above_80_F'] = reduced[list(reduced.filter(regex='days_above_'))].sum(axis=1)
# submission['d_below_30_F'] = submission[list(submission.filter(regex='days_below_'))].sum(axis=1)
# submission['d_above_80_F'] = submission[list(submission.filter(regex='days_above_'))].sum(axis=1)

In [16]:
# delete the other columns
# delete_cols(reduced, days_)
# delete_cols(submission, days_)

#### Onehot encoding

In [17]:
onehot = ["facility_type", "State_Factor", "building_class"]
#"building_class"

def onehotter(dataframe, to_onehot):
    
    for i in to_onehot:
        ohe_df = pd.get_dummies(dataframe[i], prefix=i)

        # concat with original data
        dataframe = pd.concat([dataframe, ohe_df], axis=1).drop([i], axis=1)
        
    return dataframe

In [18]:
submission = onehotter(submission, onehot)
reduced = onehotter(reduced, onehot)

In [19]:
reduced.State_Factor_State_6.value_counts

<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
75752    0
75753    0
75754    0
75755    0
75756    0
Name: State_Factor_State_6, Length: 75757, dtype: uint8>

In [20]:
# save and pop submission id apart
Y_test_sub = submission["id"]
submission.pop("id")

0       75757
1       75758
2       75759
3       75760
4       75761
        ...  
9700    85457
9701    85458
9702    85459
9703    85460
9704    85461
Name: id, Length: 9705, dtype: int64

#### Evaluation

In [21]:
def RMSE(original, predicted):
    
    aggregate = 0
    
    for orig, pred in zip(original, predicted):
        aggregate += (orig - pred)**2
    
    RMSE_ = math.sqrt(1/len(original) * aggregate)
        
    print(f'RMSE: {RMSE_}')
    
    return RMSE_

## Sample model

In [22]:
path = "/Users/charlottefelius/documents/WIDS2022/WIDS/data/train.csv"

#### Split into train and test set about  30 / 70 ratio

In [23]:
X = reduced
y = X['site_eui']
y_id = X['id']
X.pop('site_eui')
X.pop('id')
X.pop('State_Factor_State_6')

0        0
1        0
2        0
3        0
4        0
        ..
75752    0
75753    0
75754    0
75755    0
75756    0
Name: State_Factor_State_6, Length: 75757, dtype: uint8

In [24]:
raise NotImplementedError()

NotImplementedError: 

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=122)

### XGboost

In [26]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [27]:
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 11, alpha = 0.2, n_estimators = 500)

# 500, 0.2, 35.24

In [28]:
# Highest score RMSE: 35.203606 40.4 on kaggle



model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 35.203606


### LightGBM

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=22)

In [ ]:
model = lgbm.LGBMRegressor(max_depth=10, learning_rate=0.2, n_estimators=2400)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
# 37.48368551936998

predicted = model.predict(X_test)

In [ ]:
RMSE(y_test, predicted)

### Catboost (Does not work on Apple M1)

In [ ]:
MODEL_MAX_DEPTH = 12
MODEL_TASK_TYPE = 'GPU'
MODEL_RL = 0.025
MODEL_EVAL_METRIC ='RMSE'
MODEL_LOSS_FUNCTION = 'RMSE'
MODEL_ESR = 10
MODEL_VERBOSE = 1000
MODEL_ITERATIONS = 28000

In [ ]:
model = CatBoostRegressor(
    verbose=1000,
    early_stopping_rounds=10,
    #random_state=41,
    random_seed=535,
    max_depth=MODEL_MAX_DEPTH,
    task_type=MODEL_TASK_TYPE,
    learning_rate=MODEL_RL,
    iterations=28000
    
)


In [ ]:
# Highest score RMSE: 38.672019

model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
raise NotImplementedError()

#### Fit testset, write to .csv

In [29]:
# Now fit on whole training set
model.fit(X, y)
predicted = model.predict(submission)

In [30]:
# Convert Preds to DataFrame 

result = pd.DataFrame(Y_test_sub)
result["site_eui"] = predicted

In [31]:
# Write to CSV file

result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/attempt27.csv', index=False, header=True)

In [ ]:
result